In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

In [2]:
pd.options.display.max_colwidth = 500

Para realizar este ejercicio, vamos a utilizar la similitud coseno.
Primero vamos a cargar los datos de train y test

In [3]:
df_train = pd.read_csv('../data/items_titles.csv')
df_test = pd.read_csv('../data/items_titles_test.csv')

In [4]:
print(f'Las dimensiones de las tablas de train y test son: ' + str(df_train.shape) + ' y ' + str(df_test.shape))

Las dimensiones de las tablas de train y test son: (30000, 1) y (10000, 1)


Revisamos los datos de las tablas de train y test

In [5]:
df_train.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Vermelho
1,Tenis Para Caminhada Super Levinho Spider Corrida
2,Tênis Feminino Le Parc Hocks Black/ice Original Envio Já
3,Tênis Olympikus Esportivo Academia Nova Tendência Triunfo
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicleta Carregáve


In [6]:
df_test.head()

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


In [7]:
### Removemos caracteres especiales en train

df_train['ITE_ITEM_TITLE'] = df_train['ITE_ITEM_TITLE'].str.replace('[^\w\s]','')

In [8]:
## Removemos duplicados
df_train = df_train.drop_duplicates()

Los titulos de los items estan en portugues, por eso utilizamos stopwords de ese idioma. 

In [9]:
portuguese_stopwords = stopwords.words('portuguese')

In [10]:
##Instanciamos el Vectorizador
tdidf = TfidfVectorizer(stop_words=portuguese_stopwords)

In [11]:
##Creamos el vector a partir de los datos de train
vector = tdidf.fit_transform(df_train.ITE_ITEM_TITLE.to_list())

In [12]:
##Guardamos el vocabulario aunque no es necesario hacerlo en este caso
vocab = tdidf.get_feature_names()

In [13]:
##Transformamos los items del conjunto de test utilizando el objeto ya creado a partir del conjunto de entrenamiento
test_matrix = tdidf.transform(df_test['ITE_ITEM_TITLE'])

In [14]:
## Calculamos la similitud coseno entre los items del conjunto de test
similarity_matrix_test = cosine_similarity(test_matrix)

In [15]:
similarity_matrix_test.shape

(10000, 10000)

In [16]:
##Se convierte a dataframe

df_similitud = pd.DataFrame(similarity_matrix_test, 
                            columns=df_test['ITE_ITEM_TITLE'], 
                            index=df_test['ITE_ITEM_TITLE'])

In [17]:
##Se elimina la matriz de similitud de test para reducir los recursos utilizados
del similarity_matrix_test

In [18]:
##Se renombran los axis de columnas y filas para poder hacer un stack y evitar errores con los nombres de las columnas
df_similitud = df_similitud.rename_axis('ITE_ITEM_TITLE1', axis=0)
df_similitud = df_similitud.rename_axis('ITE_ITEM_TITLE2', axis=1)

In [19]:
##Reemplazamos la diagonal inferior por nulos para poder obtener valores únicos de cruce de items
##De esta forma eliminamos los casos donde el item 1 y el item 2 son los mismos o los casos duplicados (donde cambia
## el orden de los items pero el score es el mismo)
m, n = df_similitud.shape
df_similitud[:] = np.where(np.arange(m)[:,None] >= np.arange(n),np.nan,df_similitud)

In [20]:
##Se hace un stack y se renombra la columna
df_similitud = df_similitud.stack().reset_index()
df_similitud = df_similitud.rename(columns={0:'score_similitud'})

In [21]:
##Se reordenan los valores de manera descendente
df_similitud = df_similitud.sort_values('score_similitud', ascending=False)

In [22]:
##Se muestran los primeros 20 resultados
df_similitud.head(20)

,ITE_ITEM_TITLE1,ITE_ITEM_TITLE2,score_similitud
31536644,Tênis Feminino Activtta Cataluna Corrida Esportivo 4811,Tênis Corrida Activtta Feminino Cataluna Esportivo 4811-204,1.0
29898207,Caloi Explorer Comp 2021 18v Alivio,Caloi Explorer Comp 18v Alivio 2021,1.0
11297721,Tênis Feminino Torricella Preto,Tênis Feminino Preto 1579a - Torricella,1.0
23926224,Tênis Infantil Masculino Molekinho,Tênis Molekinho Infantil Masculino 260910318718,1.0
16660149,Tênis Feminino Kolosh C1323a,Tênis Feminino Kolosh Arya,1.0
6184413,Tênis - Schutz,Tênis Schutz.,1.0
37336476,Sapatenis Masculino Da Polo Lançamento 30% Off,Sapatenis Masculino Da Polo Lançamento 30% Off,1.0
16660208,Tênis Feminino Kolosh C1323a,Tênis Kolosh Worthy Feminino - K8485,1.0
10835915,Tênis Hocks - Flat Lite Tan,Tênis Hocks Flat Lite Tan Gl14,1.0
16929244,Tênis Modare Ultraconforto,Tênis Modare Ultraconforto Geltch,1.0
